In [14]:
import sys

sys.path.append(
    "C:\\Users\\Wyss User\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages"
)

import biolib
import gzip
import os

import pandas as pd

In [20]:
ALL_FASTA_OUTPUT_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\all_fasta_ht.fasta"
ALL_TARGETS_OUTPUT_DIRECTORY = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\tmhmm_all_targets_ht_panel"
ASSAY_LIST_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\ht_panel_assay_list.xlsx"
TMHMM_OUTPUT = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\tmhmm_all_targets_ht_panel\/TMRs.gff3"
UNIPROT_TO_FASTA_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\uniprotkb_proteome_UP000005640_2023_11_20.fasta.gz"

In [16]:
assay_list = pd.read_excel(ASSAY_LIST_PATH)


# Map UniProt IDs to the corresponding FASTA sequences


def parse_gz_file(file_path):
    protein_dict = {}
    current_uniprot_id = None
    current_sequence = ""
    with gzip.open(file_path, "rt") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if current_uniprot_id is not None:
                    protein_dict[current_uniprot_id] = current_sequence
                    current_sequence = ""
                if "|" in line:
                    current_uniprot_id = line.split("|")[1].strip()
                else:
                    print(f"Skipping line without expected format: {line}")
                    current_uniprot_id = None
            else:
                current_sequence += line
        if current_uniprot_id is not None:
            protein_dict[current_uniprot_id] = current_sequence
    return protein_dict

protein_data = parse_gz_file(UNIPROT_TO_FASTA_PATH)

assay_list["Sequence"] = assay_list["UniProt ID"].map(
    lambda x: protein_data.get(x, "N/A")
)

In [18]:
# Create a FASTA file containing the information for the OLINK targets

def export_fasta(output_path, df):
    with open(output_path, "w") as output_file:
        for index, row in df.iterrows():
            output_file.write(f'>{row["UniProt ID"]}\n')
            output_file.write(f'{row["Sequence"]}\n')
export_fasta(ALL_FASTA_OUTPUT_PATH, assay_list)

# Use the TMHMM deep learning model to predict the localication of each of the OLINK targets
deeptmhmm = biolib.load("DTU/DeepTMHMM")
def tmhmm_localization(targets, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    os.chdir(output_directory)
    with open("query.fasta", "w") as fasta_file:
        for _, row in targets.iterrows():
            sequence = row["Sequence"]
            uniprot_id = row["UniProt ID"]
            fasta_line = f">{uniprot_id}\n{sequence}\n"
            fasta_file.write(fasta_line)
        biolib.utils.STREAM_STDOUT = True
        deeptmhmm_job = deeptmhmm.cli(
            args="--fasta query.fasta", machine="local"
        )
        deeptmhmm_job.save_files(output_directory)

tmhmm_localization(assay_list, ALL_TARGETS_OUTPUT_DIRECTORY)

2023-12-20 14:29:06,731 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
2023-12-20 14:29:09,013 | INFO : Job "77be513e-e398-48c8-997a-a00bcd733552" running...
Running DeepTMHMM on 5397 sequences...
Step 1/4 | Loading transformer model...

Step 2/4 | Generating embeddings for sequences...
Generating embeddings: 100% 5397/5397 [6:05:21<00:00,  4.06s/seq]  

Step 3/4 | Predicting topologies for sequences in batches of 1...
Topology prediction: 100% 5397/5397 [4:37:59<00:00,  3.09s/seq]  

Step 4/4 | Generating output...
2023-12-21 01:13:24,738 | INFO : Saving 3 files to C:\Users\Wyss User\Documents\EVs\OLINK\tmhmm_all_targets_ht_panel...


In [21]:
# Import the TMHMM output
with open(TMHMM_OUTPUT, "r") as gff_file:
    gff_lines = gff_file.readlines()

# Create lists to store data
uniprot_ids = []
region_locations = []
region_starts = []
region_ends = []

# Process each line in the GFF file
for line in gff_lines:
    # Skip lines starting with "//"
    if not line.startswith("//"):
        # Split the line by tabs
        columns = line.strip().split("\t")

        # Ensure that the line has enough columns
        if len(columns) >= 4:
            # Extract information from the columns
            uniprot_id = columns[0]
            region_location = columns[1]
            region_start = int(columns[2])
            region_end = int(columns[3])

            uniprot_ids.append(uniprot_id)
            region_locations.append(region_location)
            region_starts.append(region_start)
            region_ends.append(region_end)

# Create a new dataframe
data = {
    "UniProt ID": uniprot_ids,
    "Region Location": region_locations,
    "Region Start": region_starts,
    "Region End": region_ends,
}

tmhmm_df = pd.DataFrame(data)

tm_tmhmm = tmhmm_df[tmhmm_df["Region Location"] == "TMhelix"]
outside_tmhmm = tmhmm_df[tmhmm_df["Region Location"] == "outside"]
inside_tmhmm = tmhmm_df[tmhmm_df["Region Location"] == "inside"]

tm_uniprots = set(tm_tmhmm["UniProt ID"])
inside_uniprots = set(inside_tmhmm["UniProt ID"])
outside_uniprots = set(outside_tmhmm["UniProt ID"])

inside_uniprots = inside_uniprots - tm_uniprots - outside_uniprots
outside_uniprots = outside_uniprots - tm_uniprots - inside_uniprots

In [22]:
len(tm_uniprots)

953

In [24]:
len(inside_uniprots)

3522

In [25]:
len(outside_uniprots)

922